In [ ]:
import pandas as pd
import datetime
import requests
import webbrowser
from time import sleep
from pandas.io.json import json_normalize
from IPython.display import display, HTML

youtube_alert = 'https://www.youtube.com/watch?v=io-oreIAuTM&ab_channel=MinistryofHealth%26FamilyWelfare'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}

In [ ]:
def get_states():
    states = requests.get('https://cdn-api.co-vin.in/api/v2/admin/location/states', headers=headers).json()['states']
    return pd.DataFrame(states)

In [ ]:
def get_districts(x):
    state_id = x.state_id
    state_name = x.state_name
    data = requests.get(f'https://cdn-api.co-vin.in/api/v2/admin/location/districts/{state_id}', headers=headers).json()['districts']
    districts = pd.DataFrame(data)
    districts['state_id'] = state_id
    districts['state_name'] = state_name
    return districts

In [ ]:
def run_once_a_day():
    base_date = datetime.datetime.today()
    date_list = [(base_date + datetime.timedelta(days=7*x)).strftime('%d-%m-%Y') for x in range(5)]

    try:
        states = get_states()
        temp = states.apply(lambda x: get_districts(x), axis = 1)
        all_district = pd.concat([temp[i] for i in range(0,len(temp))], ignore_index=True)
    except:
        raise Exception('Link not working - Try after 10 Mins')

    return date_list, all_district

In [ ]:
def find_my_district(name):
    temp = all_district[all_district.district_name.str.lower().str.startswith(str(name).lower())].reset_index(drop=True).copy()
    temp.drop(columns = 'state_id', inplace=True)
    return temp

In [ ]:
def get_data(district_id, date):
    try:
        data = requests.get(f"https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={district_id}&date={date}", headers=headers).json()
        df = pd.json_normalize(data=data['centers'], record_path='sessions', meta=['name','address','district_name','pincode'],errors='ignore')
    except:
        raise Exception('Link not working - Try after 10 Mins')
    
    return df

In [ ]:
def check_availability(age_limit , district_list, minimum_availability):

    df = pd.concat([get_data(district_id, date) for date in date_list for district_id in district_list], ignore_index=True)
    if df.shape[0] > 0:
        df.drop(columns=['session_id','slots'], axis=1, inplace=True)
        df.date = pd.to_datetime(df.date, format='%d-%m-%Y')
        available = df[(df.min_age_limit == age_limit) & (df.available_capacity >= minimum_availability)].sort_values(by=['date','available_capacity']).reset_index(drop=True)
        return available
    else: return pd.DataFrame()

In [ ]:
date_list, all_district = run_once_a_day()